## WindNinja CONUS Spatial Library

Creates spatial library of WindNinja simulations at 240m that match the NARR 32km grid dimensions

W. Matt Jolly, USFS, RMRS, Missoula Fire Sciences Laboratory
25 Sept 2024

In [ ]:
import ipyparallel as ipp
import requests,os,math, rioxarray, timeit
import xarray as xr
import numpy as np
from osgeo import gdal
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
from Download import *
import contextily as cx
from rioxarray.merge import merge_arrays

In [ ]:
#narropath = 'E:/Data/NARR/Data/'
bndryfile = 'WindNinjaData/ne_10m_admin_1_states_provinces_lakes.shp'  # From Natural Earth
WindNinja_cli = 'C:/WindNinja/WindNinja-3.11.0/bin/WindNinja_cli.exe'  # Installed as part of WindNinja desktop
conus_dem_file = 'WindNinjaData/LC20_Elev_220_RS_240m.tif'             # Resamples from the LANDFIRE 2020 DEM
## This is where all the outputs will be written
basedir = 'E:/Data/WindNinja/CONUS/'                                   # Set this to anything, must create the parent directory (CONUS in this example)

In [ ]:
# Function" MakeAndCallWindNinjaCLICmd
# Purpose: Form the command for a call to the WindNinja CLI and spawn a system call to it
# Inputs:
## This is an example args dictionary: args = {'input_speed' : 5, 'input_direction' : 0, 'elevation_file' : dempath, 'WindNinja_cli':WindNinja_cli}
## Note: Lots of parameters are fixed inside this function 
def MakeAndCallWindNinjaCLICmd(args):
    import os
    from osgeo import gdal
    WindNinja_cli = args['WindNinja_cli']
    initialization_method = "domainAverageInitialization"
    output_wind_height = 10
    units_output_wind_height = 'm'
    elevation_file = args['elevation_file']
    input_speed = args['input_speed'] 
    input_direction = args['input_direction']
    input_speed_units = 'mph'  
    input_wind_height = 10 
    units_input_wind_height = 'm' 
    vegetation = 'grass' 
    mesh_resolution = 240 
    units_mesh_resolution = 'm' 
    write_ascii_output = 1 
    num_threads = 1
    cmd = f'{WindNinja_cli} --elevation_file {elevation_file} --initialization_method {initialization_method} --output_wind_height {output_wind_height}\
        --units_output_wind_height {units_output_wind_height} --input_speed {input_speed} --input_speed_units {input_speed_units} --input_direction {input_direction}\
        --input_wind_height {input_wind_height} --units_input_wind_height {units_input_wind_height} --vegetation {vegetation} --mesh_resolution {mesh_resolution}\
        --units_mesh_resolution {units_mesh_resolution} --write_ascii_output {write_ascii_output} --num_threads {num_threads}'
    os.system(cmd)
    return 0

In [ ]:
bnd = gpd.read_file(bndryfile, engine='pyogrio', use_arrow=True)
bnd = bnd[bnd['gu_a3'] == 'USA']
bnd = bnd[(bnd['name'] != "Alaska")]
bnd = bnd[(bnd['name'] != "Hawaii")]

In [ ]:
#mtbnd = bnd[bnd['name'] == 'Montana']  

In [ ]:
#ax = bnd.plot()
#mtbnd.plot(ax=ax,edgecolor='red',facecolor='none')

In [ ]:
narr_proj4 = "+proj=lcc +lat_1=50 +lat_0=50 +lon_0=-107 +k_0=1 +x_0=5632642.22547 +y_0=4612545.65137 +a=6371200 +b=6371200 +units=m +no_defs"
bnd_narr_proj = bnd.to_crs(narr_proj4)    # Reproject the boundary file to the NARR projection

In [ ]:
# This is what I did to get a single WS layer from the NARR grids....
#y = 2023
#opath = narropath
#ofile = f'{opath}wsmax.10m.{y}.nc'
#vfile = f'{opath}vwnd.10m.{y}.nc'
#ufile = f'{opath}uwnd.10m.{y}.nc'
#ds_v = xr.open_dataset(vfile)
#ds_u = xr.open_dataset(ufile)  
#ds_windsp = np.sqrt(ds_u.uwnd**2 + ds_v.vwnd**2).to_dataset(name='wsmax').resample(time='1D').max('time')
#ds_windsp.wsmax[180].plot()
#ds_windsp.rio.write_crs(narr_proj4, inplace=True)
#ds_windsp.wsmax[180].rio.to_raster('narr_full_grid.tif')

In [ ]:
ds_narr_grid = xr.open_dataset('WindNinjaData/narr_full_grid.tif')
# Plot the sample NARR grid
#ds_narr_grid.band_data[0].plot()

In [ ]:
# Create a bufferd polygon for the project area (CONUS in this case)
bnd_narr_proj_buff = bnd_narr_proj.buffer(32000)
#bnd_narr_proj_buff.plot()

In [ ]:
narr_clipped = ds_narr_grid.band_data[0].rio.clip(bnd_narr_proj_buff.geometry.values, bnd_narr_proj_buff.crs, drop=True,all_touched=True)
narr_clipped.rio.to_raster('WindNinjaData/narr_mask_new.tif')

In [ ]:
f,ax = plt.subplots()
narr_clipped.plot(ax=ax)
bnd_narr_proj.plot(ax=ax,facecolor='none',edgecolor='black')

## I used QGIS to convert the pixels to polygons and I wrote that out as a shapefile called 'narr_mask.shp'.

In [ ]:
narr_grid_df = gpd.read_file('WindNinjaData/narr_mask.shp', engine='pyogrio', use_arrow=True)

In [ ]:
narr_grid_df

In [ ]:
#f,ax=plt.subplots(figsize=[12,4])
#arr_grid_df.plot(ax=ax,facecolor='none',edgecolor='black')

In [ ]:
# Read the LANDFIRE DEM resampled to 240m using Nearest Neighbor
ds_dem = xr.open_mfdataset(conus_dem_file,parallel=True)

In [ ]:
#narr_grid_df[7663:7667].plot(facecolor='none')

In [ ]:
# Clip to the Montana boundary for display and mapping example
xmin, ymin, xmax, ymax = -116.05114089810897,44.35832834237342,-104.04136319773197,49.00154597004969
narr_grid_df_sub = narr_grid_df.cx[xmin:xmax, ymin:ymax]

In [ ]:
# Extract an example polygon for a small point around Missoula, MT
#xmin, ymin, xmax, ymax = -114.4,46.8721,-114.3,46.88
#subpoly = narr_grid_df_sub.cx[xmin:xmax, ymin:ymax]

In [ ]:
## Select a subpolygon by an index (802) covers Missoula to test the process
subpoly = narr_grid_df[narr_grid_df.index == 802]
# The grid is in WGS84 Geographic, and a pixel is ~ 0.3 degrees, so let's use that to create a buffer around our pixel
subpolybuf = subpoly.buffer(0.3, join_style='mitre')
print(subpolybuf.bounds)
# Store the bounds of the buffered simulation polygon
subpolybufbnds = subpolybuf.bounds
# Let's reproject this buffered bounding box to UTM (Zone 12N in this example)
subpoly_utm = subpoly.to_crs("EPSG:32612")
# Clip the DEM. Note: Original DEM is in the LANDFIRE project, this just uses a transform from the Geographic BBOX to clip it
clipped_dem = ds_dem.rio.clip_box(minx=subpolybufbnds.minx, miny=subpolybufbnds.miny, maxx=subpolybufbnds.maxx, maxy=subpolybufbnds.maxy, crs="EPSG:4326")
# Note: Need to look up the UTM zones here from the subpoly centroid
# Reproject the DEM to UTM
clipped_dem_utm  = clipped_dem.rio.reproject("EPSG:32612")
#clipped_dem.rio.crs
# Buffer the boundary by 1/2 pixel width
subpolybufbnds = subpoly_utm.buffer(16000, join_style='mitre').bounds
# Clip the DEM to the bounding box
clipped_dem = clipped_dem_utm.rio.clip_box(minx=subpolybufbnds.minx, miny=subpolybufbnds.miny, maxx=subpolybufbnds.maxx,maxy=subpolybufbnds.maxy)


In [ ]:
ax = narr_grid_df_sub.plot(facecolor='none',edgecolor='black', figsize=(9, 9))
cx.add_basemap(ax, crs=narr_grid_df_sub.crs)
bnd.plot(ax=ax,facecolor='none',edgecolor='blue')
subpoly.plot(ax=ax,facecolor='none',edgecolor='red')

In [ ]:
# Plot the DEM in the native LANDFIRE projection
f,ax = plt.subplots()
clipped_dem.band_data[0].plot(ax=ax)
subpoly.to_crs(clipped_dem.rio.crs).plot(ax=ax,facecolor='none', edgecolor='black')
subpolybuf.to_crs(clipped_dem.rio.crs).plot(ax=ax,facecolor='none', edgecolor='black')
ax.set_title("DEM in native LANDFIRE projection")

In [ ]:
f,ax = plt.subplots()
clipped_dem_utm.band_data[0].plot(ax=ax)
subpoly_utm.buffer(16000, join_style='mitre').plot(ax=ax,facecolor='none',edgecolor='black')
ax.set_title("DEM in UTM and the NARR pixel bounding box")

In [ ]:
f,ax = plt.subplots()
clipped_dem.band_data[0].plot(ax=ax)
subpoly_utm.plot(ax=ax,facecolor='none')
subpoly_utm.buffer(16000, join_style='mitre').plot(ax=ax,facecolor='none')
ax.set_title("Final DEM for WindNinja in UTM")

## Iterate through the calls to WindNinja and map the calls to an IPyParallel cluster and view
#### On my system, there are some errors looking for GDAL library files but it doesn't seem to impact the simulation outputs

In [ ]:
# Grab the index for a polygon
#simnum = int(subpoly.index.values[0])

In [ ]:
def RunWindNinjaNARRCell(narrgridpoly,index,utm_proj="EPSG:32612"):
 
    # Grab the index this for this polygon
    simnum = index
    simnumstr = str(simnum).zfill(7)
    subpoly = narrgridpoly[narrgridpoly.index == index]
    # The grid is in WGS84 Geographic, and a pixel is ~ 0.3 degrees, so let's use that to create a buffer around our pixel
    subpolybuf = subpoly.buffer(0.3, join_style='mitre')
    print(subpolybuf.bounds)
    # Store the bounds of the buffered simulation polygon
    subpolybufbnds = subpolybuf.bounds
    # Let's reproject this buffered bounding box to UTM (Zone 12N in this example)
    subpoly_utm = subpoly.to_crs(utm_proj)
    # Clip the DEM. Note: Original DEM is in the LANDFIRE project, this just uses a transform from the Geographic BBOX to clip it
    clipped_dem = ds_dem.rio.clip_box(minx=subpolybufbnds.minx, miny=subpolybufbnds.miny, maxx=subpolybufbnds.maxx, maxy=subpolybufbnds.maxy, crs="EPSG:4326")
    # Note: Need to look up the UTM zones here from the subpoly centroid
    # Reproject the DEM to UTM
    clipped_dem_utm  = clipped_dem.rio.reproject(utm_proj)
    #clipped_dem.rio.crs
    # Buffer the boundary by 1/2 pixel width
    subpolybufbnds = subpoly_utm.buffer(16000, join_style='mitre').bounds
    # Clip the DEM to the bounding box
    clipped_dem = clipped_dem_utm.rio.clip_box(minx=subpolybufbnds.minx, miny=subpolybufbnds.miny, maxx=subpolybufbnds.maxx,maxy=subpolybufbnds.maxy)
    
    # Create the output path
    path = f'{basedir}{simnumstr}/'
    dempath = f'{path}{simnumstr}_dem.tif'
    print(dempath)
    
    # Save the DEM to the simulation directory
    clipped_dem.band_data[0].rio.to_raster(dempath)
    
    # Check whether the specified path exists or not
    isExist = os.path.exists(path)
    if not isExist:
      # Create a new directory because it does not exist 
      os.makedirs(path)
      print("The new data directory is created!")
        
    arglist = []  # A list of all the individual simulations for WindNinja
    nproc = 8
    # Create a cluster with 8 processors
    with ipp.Cluster(n=nproc) as rc:
        # Create an array of the windspeed ranges
        wdirrange = np.arange(0,360,45)
        # get a view on the cluster
        view = rc.load_balanced_view()    
        for wd in wdirrange:
            # Create the input args to to the WindNinja function
            args = {'input_speed' : 5, 'input_direction' : wd, 'elevation_file' : dempath, 'WindNinja_cli':WindNinja_cli}
            # Append the args to a simulation list
            arglist.append(args)
        # submit the tasks to the Cluster
        # Map the list of function inputs to the function calls
        syncresult = view.map(MakeAndCallWindNinjaCLICmd, arglist)
        # wait for the threads to finish
        syncresult.wait_interactive()
        # retrieve actual results
        result = syncresult.get()
        

In [ ]:
# Run the WindNinja libraries for 3 cells, these are Missoula and the two cells West
RunWindNinjaNARRCell(narr_grid_df,800)
RunWindNinjaNARRCell(narr_grid_df,801)
RunWindNinjaNARRCell(narr_grid_df,802)

## Let's look at some results from the 3 different NARR grid cells

In [ ]:
# Let's load three North Wind scenarios @ 5 mph for adjacent pixels
f = f'{basedir}0000800/0000800_dem_0_5_240m_vel.asc'
ds1 = xr.open_dataset(f)
f = f'{basedir}0000801/0000801_dem_0_5_240m_vel.asc'
ds2 = xr.open_dataset(f)
f = f'{basedir}0000802/0000802_dem_0_5_240m_vel.asc'
ds3 = xr.open_dataset(f)

In [ ]:
subpoly1 = narr_grid_df[narr_grid_df.index == 800].to_crs("EPSG:32612")
subpoly2 = narr_grid_df[narr_grid_df.index == 801].to_crs("EPSG:32612")
subpoly3 = narr_grid_df[narr_grid_df.index == 802].to_crs("EPSG:32612")

In [ ]:
#ax=subpoly1.plot(facecolor='none',edgecolor='black')
#subpoly1.buffer(60).plot(facecolor='none',edgecolor='black',ax=ax)
#subpoly2.plot(facecolor='none',edgecolor='black',ax=ax)
#subpoly3.plot(facecolor='none',edgecolor='black',ax=ax)

## Clip the larger simulation down to the NARR pixel geometry with a 60m buffer to avoid striping 

In [ ]:
# If you don't buffer these images a little before clipping, you get seams of non-data in between. This seems to fix that.
# This clips
ds_clipped1 = ds1.rio.clip(subpoly1.buffer(60).geometry.values,subpoly1.crs, drop=True, invert=False).transpose('band', 'y', 'x')
ds_clipped2 = ds2.rio.clip(subpoly2.buffer(60).geometry.values,subpoly2.crs, drop=True, invert=False).transpose('band', 'y', 'x')
ds_clipped3 = ds3.rio.clip(subpoly3.buffer(60).geometry.values,subpoly3.crs, drop=True, invert=False).transpose('band', 'y', 'x')

In [ ]:
# Merge/Mosaic multiple rasters using merge_arrays method of rioxarray
merged_raster = merge_arrays(dataarrays = [ds_clipped1.band_data[0],ds_clipped2.band_data[0],ds_clipped3.band_data[0]], res = (240, 240), crs="EPSG:32612", nodata = -999.9)
 

In [ ]:
f,ax = plt.subplots()
merged_raster.rio.reproject("EPSG:5070").plot(levels=[0,1,2,3,4,5,6,7,8,9,10,12,15,18],cmap='RdYlBu_r',ax=ax)
subpoly1.to_crs("EPSG:5070").plot(ax=ax,facecolor='none',edgecolor='black')
subpoly2.to_crs("EPSG:5070").plot(ax=ax,facecolor='none',edgecolor='black')
subpoly3.to_crs("EPSG:5070").plot(ax=ax,facecolor='none',edgecolor='black')
ax.set_title('North Wind @ 5mph for 3 NARR cells')